In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [28]:
dataset = pd.read_csv('insurance.csv')

In [29]:
dataset.head()

,108,392.5
0,19,46.2
1,13,15.7
2,124,422.2
3,40,119.4
4,57,170.9


In [30]:
#Classe da regressão linear univariada
class simple_linear_regression(object):
    
    def __init__(self):
        self.b0 = None
        self.b1 = None
    
    def fit(self,train):
        self.b0, self.b1 = self.coefficients(train)
        
    def predict(self, test):
        predictions = []
        for row in test:
            ypred = self.b0 + self.b1 * row
            predictions.append(ypred)
        return predictions
    
    # Calculate coefficients
    def coefficients(self, dataset):
        x = dataset.iloc[:,0]
        y = dataset.iloc[:,1]
        x_mean, y_mean = self.mean(x), self.mean(y)
        b1 = self.covariance(x, x_mean, y, y_mean) / self.variance(x, x_mean)
        b0 = y_mean - b1 * x_mean
        return [b0, b1]
    
    # Calculate covariance between x and y
    def covariance(self, x, mean_x, y, mean_y):
        covar = 0.0
        for i in range(len(x)):
            covar += (x.iloc[i] - mean_x) * (y.iloc[i] - mean_y)
        return covar
    
    # Calculate the variance of a list of numbers
    def variance(self, values, mean):
        return sum([(x-mean)**2 for x in values])

    # Calculate the mean value of a list of numbers
    def mean(self, values):
        return sum(values) / float(len(values))


In [31]:


#Separa o dataset em teste e treino
data_train, data_test = train_test_split(dataset, test_size=0.40)
#Separa os dados de teste em X e y
X_test = data_test.iloc[:,0]
y_test = data_test.iloc[:,1]
y_list = y_test.tolist()



In [32]:


#Executa a regressão linear e prediz os valores para X_test
lr = simple_linear_regression()
lr.fit(data_train)
predict = lr.predict(X_test)



In [33]:


from math import sqrt

# Calculate root mean squared error
def rmse_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        prediction_error = predicted[i] - actual[i]
        sum_error += (prediction_error ** 2)
        mean_error = sum_error / float(len(actual))
    return sqrt(mean_error)

In [34]:
#Calcula o RMSE para o algoritmo de regressão
rmse_reg = rmse_metric(y_list,predict)


In [35]:
import bisect as bs

#Definição da função smart_mean
#Essa função faz uma aproximação linear entre os dois valores mais próximos, no conjunto de treino, 
#em relação a amostra de teste
def smart_mean(data_train,test):
    data_sort = data_train.sort_values(data_train.columns[0], ascending = True)
    X_train = (data_sort.iloc[:,0]).tolist()
    y_train = (data_sort.iloc[:,1]).tolist()
    predict = []
    for x in test:
        bigger = bs.bisect_right(X_train,x)
        smaller = bs.bisect_left(X_train,x)
        if(bigger > len(X_train)-1):
            valuePredict = (x*y_train[smaller-1])/X_train[smaller-1]
        else:
            dif_x = (X_train[bigger] - X_train[smaller])
            if(dif_x != 0):
                incValue = (y_train[bigger] - y_train[smaller]) / dif_x
                valuePredict = y_train[smaller] + dif_x*incValue
            else:
                valuePredict = y_train[smaller]
        predict.append(valuePredict)
    return predict

#Definição da função baseline, prediz que todos os valores são a média dos dados de treino
def baseline(y_train, test):
    meanValue = np.mean(y_train)
    predictions = [meanValue for i in range(len(test))]
    return predictions

In [36]:
#Calcula o RMSE para o algoritmo de smart_mean
predict_smartmean = smart_mean(data_train,X_test)
rmse_smart = rmse_metric(y_list,predict_smartmean)

In [37]:
#Calcula o RMSE para o algoritmo baseline
predict_base = baseline(y_list,X_test)
rmse_baseline = rmse_metric(y_list,predict_base)

In [38]:
print('Predições')
for i in range(len(y_list)):
    print("valor real: {:.3f}\t|| regressão: {:.3f}\t|| smart_mean: {:.3f}\t|| baseline: {:.3f}".format(y_list[i], predict[i], predict_smartmean[i], predict_base[i]))


Predições
valor real: 0.000	|| regressão: 14.949	|| smart_mean: 6.600	|| baseline: 99.652
valor real: 69.200	|| regressão: 103.054	|| smart_mean: 92.600	|| baseline: 99.652
valor real: 23.500	|| regressão: 53.715	|| smart_mean: 15.700	|| baseline: 99.652
valor real: 142.100	|| regressão: 74.861	|| smart_mean: 161.500	|| baseline: 99.652
valor real: 119.400	|| regressão: 155.917	|| smart_mean: 73.400	|| baseline: 99.652
valor real: 194.500	|| regressão: 120.675	|| smart_mean: 152.800	|| baseline: 99.652
valor real: 98.100	|| regressão: 85.433	|| smart_mean: 161.500	|| baseline: 99.652
valor real: 27.900	|| regressão: 39.619	|| smart_mean: 87.400	|| baseline: 99.652
valor real: 31.900	|| regressão: 60.764	|| smart_mean: 77.500	|| baseline: 99.652
valor real: 209.800	|| regressão: 124.200	|| smart_mean: 152.800	|| baseline: 99.652
valor real: 65.300	|| regressão: 50.191	|| smart_mean: 21.300	|| baseline: 99.652
valor real: 59.600	|| regressão: 71.336	|| smart_mean: 161.500	|| baseline: 99

In [39]:


print('RMSE para regressão linear: {0}'.format(rmse_reg))
print('RMSE para smart_mean: {0}'.format(rmse_smart))
print('RMSE para baseline: {0}'.format(rmse_baseline))



RMSE para regressão linear: 37.55960335298851
RMSE para smart_mean: 50.719209425854224
RMSE para baseline: 88.22677199127259
